<h2><b>Identification of important events and entities from the document

<b>Import requried libraries

In [106]:
import pandas as pd
import string
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.frequent_patterns import apriori,association_rules
from sklearn.metrics.pairwise import cosine_similarity

stopword = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

<b>Load twitter dataset from kaggle

In [87]:
df = pd.read_csv('./609430_1124303_compressed_2020-04-16 Coronavirus Tweets.CSV/2020-04-16 Coronavirus Tweets.CSV')

In [90]:
df = df[['created_at','text','lang']]

<b>Extract #covid19 data from number of hashtag's

In [5]:
covid = df[df['text'].apply(lambda x:'#covid19'in x)].iloc[20000:21000,:]

In [6]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

def clean_textWithoutSteam(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [word for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text

<b>Create Dataframe of DTM of with stemming and without stemming

In [107]:
def countVect(text):
    countVectorizer = CountVectorizer(analyzer=clean_textWithoutSteam) 
    countVector = countVectorizer.fit_transform(covid['text'])
    dtm = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
    
    countVectorizer1 = CountVectorizer(analyzer=clean_text) 
    countVector1 = countVectorizer1.fit_transform(covid['text'])
    dtm1 = pd.DataFrame(countVector1.toarray(), columns=countVectorizer1.get_feature_names())
    
    return (dtm,dtm1)

df_withoutStem, df_withStem = countVect(covid['text'])

<b> Get Association of two dataframe's

In [108]:
def getAssociation(dtm1,dtm):
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    word_sets_withStem = dtm1.iloc[:,1:].applymap(encode_units)
    word_sets_withoutStem = dtm.iloc[:,1:].applymap(encode_units)
    
    prob_words_withStem = apriori(word_sets_withStem,min_support=0.01,verbose=1,use_colnames=True)
    prob_words_withoutStem = apriori(word_sets_withoutStem,min_support=0.01,verbose=1,use_colnames=True)
    
    rules_withStem = association_rules(prob_words_withStem,min_threshold=0)
    rules_withoutStem = association_rules(prob_words_withoutStem,min_threshold=0)
    
    return(rules_withStem,rules_withoutStem)

In [109]:
rules_withStem, rules_withoutStem = getAssociation(df_withStem,df_withoutStem)

Processing 6 combinations | Sampling itemset size 6 543
Processing 6 combinations | Sampling itemset size 6 543


In [98]:
rules_withStem.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(al),(covid),0.017,0.993,0.017,1.000000,1.007049,0.000119,inf
1,(covid),(al),0.993,0.017,0.017,0.017120,1.007049,0.000119,1.000122
2,(amp),(covid),0.053,0.993,0.053,1.000000,1.007049,0.000371,inf
3,(covid),(amp),0.993,0.053,0.053,0.053374,1.007049,0.000371,1.000395
4,(april),(covid),0.016,0.993,0.016,1.000000,1.007049,0.000112,inf


In [99]:
rules_withoutSteam.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(al),(covid),0.016,0.993,0.016,1.000000,1.007049,0.000112,inf
1,(covid),(al),0.993,0.016,0.016,0.016113,1.007049,0.000112,1.000115
2,(amp),(covid),0.053,0.993,0.053,1.000000,1.007049,0.000371,inf
3,(covid),(amp),0.993,0.053,0.053,0.053374,1.007049,0.000371,1.000395
4,(april),(covid),0.014,0.993,0.014,1.000000,1.007049,0.000098,inf


In [17]:
asso_withSteam = rules_withStem[['antecedents','consequents','support','confidence']]
asso_withoutSteam = rules_withoutSteam[['antecedents','consequents','support','confidence']]

asso_withSteam.columns = pd.MultiIndex.from_product([['With_Stemming'], asso_withSteam.columns])
asso_withoutSteam.columns = pd.MultiIndex.from_product([['Without_Stemming'], asso_withoutSteam.columns])

result = pd.concat([asso_withSteam, asso_withoutSteam], axis = 1)

In [101]:
result.head()
#support = Probability
#confidence = Conditinal Probability

With_Stemming                                Without_Stemming              \
    antecedents consequents support confidence      antecedents consequents   
0          (al)     (covid)   0.017   1.000000             (al)     (covid)   
1       (covid)        (al)   0.017   0.017120          (covid)        (al)   
2         (amp)     (covid)   0.053   1.000000            (amp)     (covid)   
3       (covid)       (amp)   0.053   0.053374          (covid)       (amp)   
4       (april)     (covid)   0.016   1.000000          (april)     (covid)   

                      
  support confidence  
0   0.016   1.000000  
1   0.016   0.016113  
2   0.053   1.000000  
3   0.053   0.053374  
4   0.014   1.000000

<b> Cosine similarity of each documents of with stemming and without stemming

In [105]:
docs_similarity_withStem = pd.DataFrame(cosine_similarity(df_withStem, df_withStem))
docs_similarity_withStem.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.133631,0.117851,0.046225,0.045361,0.117851,0.092450,0.044544,0.171499,0.058926,...,0.051434,0.204124,0.098295,0.144338,0.035533,0.058926,0.089087,0.097129,0.114332,0.071067
1,0.133631,1.000000,0.094491,0.037062,0.036370,0.094491,0.111187,0.071429,0.091670,0.188982,...,0.041239,0.109109,0.078811,0.077152,0.028490,0.094491,0.071429,0.025959,0.045835,0.056980
2,0.117851,0.094491,1.000000,0.098058,0.096225,0.250000,0.196116,0.094491,0.121268,0.125000,...,0.109109,0.144338,0.208514,0.102062,0.075378,0.125000,0.188982,0.068680,0.121268,0.150756
3,0.046225,0.037062,0.098058,1.000000,0.113228,0.098058,0.076923,0.037062,0.047565,0.049029,...,0.042796,0.056614,0.081786,0.080064,0.029566,0.098058,0.111187,0.026939,0.047565,0.059131
4,0.045361,0.036370,0.096225,0.113228,1.000000,0.096225,0.075485,0.036370,0.093352,0.048113,...,0.041996,0.055556,0.120386,0.157135,0.029013,0.096225,0.072739,0.026435,0.046676,0.058026


In [104]:
docs_similarity_withoutStem = pd.DataFrame(cosine_similarity(df_withoutStem, df_withoutStem))
docs_similarity_withoutStem.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,0.133631,0.117851,0.046225,0.047140,0.117851,0.092450,0.044544,0.171499,0.058926,...,0.051434,0.204124,0.098295,0.144338,0.035533,0.058926,0.089087,0.097129,0.114332,0.071067
1,0.133631,1.000000,0.094491,0.037062,0.037796,0.094491,0.111187,0.071429,0.091670,0.141737,...,0.041239,0.109109,0.078811,0.077152,0.028490,0.094491,0.071429,0.025959,0.045835,0.056980
2,0.117851,0.094491,1.000000,0.098058,0.100000,0.250000,0.196116,0.094491,0.121268,0.125000,...,0.109109,0.144338,0.208514,0.102062,0.075378,0.125000,0.188982,0.068680,0.121268,0.150756
3,0.046225,0.037062,0.098058,1.000000,0.078446,0.098058,0.076923,0.037062,0.047565,0.049029,...,0.042796,0.056614,0.081786,0.080064,0.029566,0.049029,0.074125,0.026939,0.047565,0.059131
4,0.047140,0.037796,0.100000,0.078446,1.000000,0.100000,0.078446,0.037796,0.097014,0.050000,...,0.043644,0.057735,0.083406,0.122474,0.030151,0.100000,0.075593,0.027472,0.048507,0.060302
